 ## BuatToolhelp32Snapshot

CreateToolhelp32Snapshot API biasanya digunakan dalam pemrograman C++ untuk menghitung proses dan modul pada sistem Windows. Meskipun ini bukan API yang terkait langsung dengan keamanan siber atau pengujian pena, API ini dapat digunakan untuk memperoleh informasi tentang proses yang berjalan, yang dapat berguna dalam konteks keamanan.


In [ ]:
#include <windows.h>
#include <tlhelp32.h>
#include <iostream>

int main() {
    //Create a snapshot of running processes
    HANDLE hSnapshot = CreateToolhelp32Snapshot(TH32CS_SNAPPROCESS, 0);
    if (hSnapshot == INVALID_HANDLE_VALUE) {
        std::cerr << "Erro ao criar o snapshot: " << GetLastError() << std::endl;
        return 1;
    }

    // Structure for storing information about a process
    PROCESSENTRY32 pe32;
    pe32.dwSize = sizeof(PROCESSENTRY32);

    // Initialize the loop to enumerate the processes
    if (Process32First(hSnapshot, &pe32)) {
        do {
            std::cout << "Processo ID: " << pe32.th32ProcessID << std::endl;
            std::cout << "Nome do processo: " << pe32.szExeFile << std::endl;
        } while (Process32Next(hSnapshot, &pe32));
    } else {
        std::cerr << "Erro ao enumerar processos: " << GetLastError() << std::endl;
    }

    // Close the snapshot
    CloseHandle(hSnapshot);

    return 0;
}

## DapatkanModuleFileName

GetModuleFileName API di C++ biasanya digunakan untuk mengambil path lengkap dari file yang dapat dieksekusi dari proses yang sedang berjalan. Meskipun tidak terkait langsung dengan keamanan siber atau pengujian penetrasi, hal ini dapat berguna dalam bidang tersebut untuk mengumpulkan informasi tentang proses yang berjalan pada suatu sistem.

Berikut adalah contoh kode C++ sederhana yang menunjukkan cara menggunakan GetModuleFileName API untuk mengambil path lengkap dari executable untuk proses tertentu menggunakan Process ID (PID). Informasi ini dapat berguna dalam skenario audit keamanan dan pemantauan proses.

In [ ]:
#include <windows.h>
#include <iostream>

int main() {
    DWORD processId; // Replace with the target process ID
    HANDLE hProcess;

    // Replace 'processId' with the PID of the target process
    processId = 1234; // Example PID

    // Open the target process with PROCESS_QUERY_INFORMATION and PROCESS_VM_READ access rights
    hProcess = OpenProcess(PROCESS_QUERY_INFORMATION | PROCESS_VM_READ, FALSE, processId);

    if (hProcess == NULL) {
        std::cerr << "Failed to open the target process. Error code: " << GetLastError() << std::endl;
        return 1;
    }

    char szPath[MAX_PATH];
    DWORD dwSize = GetModuleFileNameExA(hProcess, NULL, szPath, MAX_PATH);

    if (dwSize == 0) {
        std::cerr << "Failed to get module filename. Error code: " << GetLastError() << std::endl;
        CloseHandle(hProcess);
        return 1;
    }

    std::cout << "Full path of the executable: " << szPath << std::endl;

    // Close the process handle
    CloseHandle(hProcess);

    return 0;
}

## ShellExecuteEx

ShellExecuteEx API di C++ biasanya digunakan untuk meluncurkan aplikasi eksternal dan melakukan berbagai operasi terkait file. Meskipun ini mungkin bukan alat langsung untuk pengujian keamanan siber atau penetrasi, ini dapat digunakan di bidang ini untuk tugas pembuatan skrip atau otomatisasi, seperti membuka file atau URL tertentu sebagai bagian dari penilaian. Berikut contoh sederhana penggunaan ShellExecuteEx untuk membuka halaman web:

In [ ]:
#include <windows.h>
#include <iostream>

int main() {
    SHELLEXECUTEINFO shellInfo = {0};
    shellInfo.cbSize = sizeof(SHELLEXECUTEINFO);
    shellInfo.fMask = SEE_MASK_NOCLOSEPROCESS;
    shellInfo.lpFile = L"https://www.example.com"; // Replace with the URL you want to open
    shellInfo.lpVerb = L"open";
    shellInfo.nShow = SW_SHOWNORMAL;

    if (ShellExecuteEx(&shellInfo)) {
        WaitForSingleObject(shellInfo.hProcess, INFINITE);
        CloseHandle(shellInfo.hProcess);
        std::cout << "Web page opened successfully!" << std::endl;
    } else {
        std::cerr << "Failed to open the web page. Error code: " << GetLastError() << std::endl;
        return 1;
    }

    return 0;


Dalam contoh ini, kami menggunakan ShellExecuteEx untuk membuka halaman web (https://www.example.com) menggunakan browser web default. Flag SEE_MASK_NOCLOSEPROCESS diatur untuk mendapatkan pegangan pada proses yang diluncurkan, dan WaitForSingleObject digunakan untuk menunggu proses selesai.

## DapatkanTokenInformasi

GetTokenInformation API di C++ digunakan untuk mengambil informasi tentang token keamanan yang terkait dengan suatu proses atau thread. Ini dapat bermanfaat dalam pengujian keamanan siber dan penetrasi ketika Anda perlu mengumpulkan informasi tentang hak istimewa, grup, atau karakteristik lain dari token akses pengguna. Berikut ini contoh cara menggunakan GetTokenInformation untuk mengambil grup tempat pengguna berada:

In [ ]:
#include <windows.h>
#include <iostream>

int main() {
    HANDLE hToken = NULL;

    if (!OpenProcessToken(GetCurrentProcess(), TOKEN_QUERY, &hToken)) {
        std::cerr << "OpenProcessToken failed. Error code: " << GetLastError() << std::endl;
        return 1;
    }

    DWORD dwSize = 0;
    GetTokenInformation(hToken, TokenGroups, NULL, 0, &dwSize);

    if (GetLastError() != ERROR_INSUFFICIENT_BUFFER) {
        std::cerr << "GetTokenInformation failed (1). Error code: " << GetLastError() << std::endl;
        CloseHandle(hToken);
        return 1;
    }

    PTOKEN_GROUPS pGroups = reinterpret_cast<PTOKEN_GROUPS>(new BYTE[dwSize]);

    if (!GetTokenInformation(hToken, TokenGroups, pGroups, dwSize, &dwSize)) {
        std::cerr << "GetTokenInformation failed (2). Error code: " << GetLastError() << std::endl;
        delete[] pGroups;
        CloseHandle(hToken);
        return 1;
    }

    std::cout << "Token Groups:" << std::endl;
    for (DWORD i = 0; i < pGroups->GroupCount; ++i) {
        SID_NAME_USE sidType;
        WCHAR szName[256];
        DWORD cchName = sizeof(szName) / sizeof(szName[0]);
        if (LookupAccountSidW(NULL, pGroups->Groups[i].Sid, szName, &cchName, NULL, NULL, &sidType)) {
            std::wcout << L"Group " << i + 1 << L": " << szName << std::endl;
        }
    }

    delete[] pGroups;
    CloseHandle(hToken);

    return 0;
}

Kami membuka token akses yang terkait dengan proses saat ini menggunakan OpenProcessToken.

Pertama-tama kita memanggil GetTokenInformation dengan buffer NULL untuk menentukan ukuran buffer yang diperlukan (dwSize). Kemudian, kami mengalokasikan memori untuk struktur informasi token berdasarkan ukuran ini.

Kami memanggil GetTokenInformation lagi untuk mengambil informasi grup token.

Kami melakukan iterasi melalui grup token dan menggunakan LookupAccountSidW untuk mengonversi SID grup menjadi nama yang dapat dibaca manusia dan menampilkannya.

## Sesuaikan Hak Istimewa Token

API AdjustTokenPrivileges di C++ digunakan untuk mengaktifkan atau menonaktifkan hak istimewa dalam token akses. Ini biasanya digunakan dalam skenario pengujian keamanan siber dan penetrasi ketika Anda perlu menyesuaikan hak istimewa untuk melakukan tindakan tertentu dengan izin yang lebih tinggi. Berikut ini contoh cara menggunakan AdjustTokenPrivileges untuk mengaktifkan hak istimewa untuk proses saat ini:

In [ ]:
#include <windows.h>
#include <iostream>

int main() {
    HANDLE hToken = NULL;

    // Open the access token for the current process with TOKEN_ADJUST_PRIVILEGES privilege
    if (!OpenProcessToken(GetCurrentProcess(), TOKEN_ADJUST_PRIVILEGES | TOKEN_QUERY, &hToken)) {
        std::cerr << "OpenProcessToken failed. Error code: " << GetLastError() << std::endl;
        return 1;
    }

    // Specify the privilege to enable (e.g., SE_DEBUG_NAME)
    LUID luid;
    if (!LookupPrivilegeValue(NULL, SE_DEBUG_NAME, &luid)) {
        std::cerr << "LookupPrivilegeValue failed. Error code: " << GetLastError() << std::endl;
        CloseHandle(hToken);
        return 1;
    }

    // Prepare the TOKEN_PRIVILEGES structure
    TOKEN_PRIVILEGES tp;
    tp.PrivilegeCount = 1;
    tp.Privileges[0].Luid = luid;
    tp.Privileges[0].Attributes = SE_PRIVILEGE_ENABLED;

    // Adjust the token privileges
    if (!AdjustTokenPrivileges(hToken, FALSE, &tp, sizeof(TOKEN_PRIVILEGES), NULL, NULL)) {
        std::cerr << "AdjustTokenPrivileges failed. Error code: " << GetLastError() << std::endl;
        CloseHandle(hToken);
        return 1;
    }

    if (GetLastError() == ERROR_NOT_ALL_ASSIGNED) {
        std::cerr << "The token does not have the specified privilege." << std::endl;
    } else {
        std::cout << "The privilege has been enabled." << std::endl;
    }

    CloseHandle(hToken);
    
    return 0;
}

## Toolhelp32ReadProcessMemory

Toolhelp32ReadProcessMemory bukan fungsi Windows API standar atau yang dikenali. Tampaknya ini adalah salah tafsir atau kombinasi dari dua fungsi terpisah, Toolhelp32Snapshot dan ReadProcessMemory, seperti yang saya sebutkan sebelumnya.

Jika Anda ingin membaca memori proses berbeda untuk tujuan keamanan siber atau pengujian penetrasi, Anda dapat menggunakan ReadProcessMemory. Berikut ini contoh cara menggunakan ReadProcessMemory untuk membaca memori proses lain:

In [ ]:
#include <windows.h>
#include <iostream>

int main() {
    DWORD processId; // Replace with the target process ID
    HANDLE hProcess;

    // Replace 'processId' with the PID of the target process
    processId = 1234; // Example PID

    // Open the target process with PROCESS_VM_READ access rights
    hProcess = OpenProcess(PROCESS_VM_READ, FALSE, processId);

    if (hProcess == NULL) {
        std::cerr << "Failed to open the target process. Error code: " << GetLastError() << std::endl;
        return 1;
    }

    // Define a buffer to store the read data
    SIZE_T bytesRead;
    DWORD address = 0x12345678; // Replace with the memory address you want to read
    DWORD buffer;

    // Read memory from the target process
    if (ReadProcessMemory(hProcess, (LPCVOID)address, &buffer, sizeof(DWORD), &bytesRead)) {
        std::cout << "Read value at address " << std::hex << address << ": " << std::dec << buffer << std::endl;
    } else {
        std::cerr << "ReadProcessMemory failed. Error code: " << GetLastError() << std::endl;
    }

    // Close the handle to the target process
    CloseHandle(hProcess);

    return 0;
}

## TulisProsesMemori

WriteProcessMemory API di C++ digunakan untuk menulis data ke memori proses lain. Ini dapat berguna dalam skenario pengujian keamanan siber dan penetrasi ketika Anda perlu memodifikasi atau memasukkan kode ke dalam proses lain. Perlu diketahui bahwa mengubah memori proses lain dapat menimbulkan implikasi hukum dan etika, dan Anda sebaiknya hanya menggunakan API ini secara bertanggung jawab dan dengan izin yang sesuai.

Berikut ini contoh cara menggunakan WriteProcessMemory untuk menulis nilai ke memori proses lain:

In [ ]:
#include <windows.h>
#include <iostream>

int main() {
    DWORD processId; // Replace with the target process ID
    HANDLE hProcess;

    // Replace 'processId' with the PID of the target process
    processId = 1234; // Example PID

    // Open the target process with PROCESS_VM_WRITE and PROCESS_VM_OPERATION access rights
    hProcess = OpenProcess(PROCESS_VM_WRITE | PROCESS_VM_OPERATION, FALSE, processId);

    if (hProcess == NULL) {
        std::cerr << "Failed to open the target process. Error code: " << GetLastError() << std::endl;
        return 1;
    }

    DWORD address = 0x12345678; // Replace with the memory address you want to write to
    DWORD value = 42; // Replace with the value you want to write

    // Write the value to the memory of the target process
    SIZE_T bytesWritten;
    if (WriteProcessMemory(hProcess, (LPVOID)address, &value, sizeof(DWORD), &bytesWritten)) {
        if (bytesWritten == sizeof(DWORD)) {
            std::cout << "Successfully wrote value " << value << " to address " << std::hex << address << std::endl;
        } else {
            std::cerr << "Partial write: " << bytesWritten << " bytes written instead of " << sizeof(DWORD) << std::endl;
        }
    } else {
        std::cerr << "WriteProcessMemory failed. Error code: " << GetLastError() << std::endl;
    }

    // Close the handle to the target process
    CloseHandle(hProcess);

    return 0;
}

## WTSEnumerateProcessesEx

WTSEnumerateProcessesEx adalah API yang digunakan untuk menghitung proses di Server Terminal Windows. Biasanya digunakan untuk tujuan administratif, bukan untuk keamanan siber atau pengujian penetrasi. Namun, ini dapat digunakan untuk mengumpulkan informasi tentang proses yang berjalan di server jarak jauh, yang mungkin relevan dengan penilaian keamanan tertentu. Untuk menggunakan API ini, Anda harus menyertakan perpustakaan wtsapi32.lib.

Berikut ini contoh cara menggunakan WTSEnumerateProcessesEx untuk membuat daftar proses di Server Terminal jarak jauh:

In [ ]:
#include <windows.h>
#include <wtsapi32.h>
#include <iostream>

int main() {
    PWTS_PROCESS_INFO_EX pProcessInfo = NULL;
    DWORD dwProcCount = 0;
    
    if (WTSEnumerateProcessesEx(WTS_CURRENT_SERVER_HANDLE, &pProcessInfo, &dwProcCount) != 0) {
        for (DWORD i = 0; i < dwProcCount; ++i) {
            std::wcout << L"Process ID: " << pProcessInfo[i].ProcessId << std::endl;
            std::wcout << L"Session ID: " << pProcessInfo[i].SessionId << std::endl;
            std::wcout << L"Process Name: " << pProcessInfo[i].pProcessName << std::endl;
            std::wcout << L"User Name: " << pProcessInfo[i].pUserSid << std::endl;
            std::wcout << L"--------------------------------------" << std::endl;
        }

        // Free the allocated memory
        WTSFreeMemory(pProcessInfo);
    } else {
        std::cerr << "WTSEnumerateProcessesEx failed. Error code: " << GetLastError() << std::endl;
        return 1;
    }

    return 0;
}

## WTSFreeMemoryEx

Saya minta maaf, tetapi pada pembaruan pengetahuan terakhir saya pada bulan September 2021, tidak ada fungsi Windows API standar yang bernama WTSFreeMemoryEx. Oleh karena itu, saya tidak dapat memberikan contoh kode C++ untuk API khusus ini.

Ada kemungkinan bahwa API semacam itu diperkenalkan di versi Windows yang lebih baru atau sebagai bagian dari perpustakaan pihak ketiga. Jika Anda memiliki informasi spesifik tentang WTSFreeMemoryEx atau tujuan penggunaannya, harap berikan detail lebih lanjut, dan saya akan melakukan yang terbaik untuk membantu Anda dengan contoh kode.

In [ ]:
BOOL WTSFreeMemoryExA(
  [in] WTS_TYPE_CLASS WTSTypeClass,
  [in] PVOID          pMemory,
  [in] ULONG          NumberOfEntries
);

## PencarianNilai Hak Istimewa

API LookupPrivilegeValue di C++ digunakan untuk mengambil pengidentifikasi unik lokal (LUID) yang mewakili nama hak istimewa pada sistem. API ini dapat berguna dalam keamanan siber dan pengujian penetrasi ketika Anda perlu bekerja dengan hak istimewa, seperti mengaktifkan atau menonaktifkannya untuk suatu proses. Berikut ini contoh cara menggunakan LookupPrivilegeValue untuk mengambil LUID untuk hak istimewa:

In [ ]:
#include <windows.h>
#include <iostream>

int main() {
    LUID luid;
    if (LookupPrivilegeValue(NULL, SE_DEBUG_NAME, &luid)) {
        std::cout << "LUID for SE_DEBUG_NAME: " << std::dec << luid.LowPart << ":" << luid.HighPart << std::endl;
    } else {
        std::cerr << "LookupPrivilegeValue failed. Error code: " << GetLastError() << std::endl;
        return 1;
    }

    return 0;
}

Kami memanggil LookupPrivilegeValue untuk mengambil LUID untuk hak istimewa bernama SE_DEBUG_NAME. Hak istimewa ini sering digunakan dalam skenario debugging dan merupakan contoh nama hak istimewa.

Jika fungsinya berhasil, ia akan mengembalikan LUID untuk hak istimewa yang ditentukan, yang terdiri dari dua bagian: LowPart dan HighPart. Kami mencetak nilai-nilai ini menggunakan std::cout.

Jika LookupPrivilegeValue gagal, kami mencetak pesan kesalahan dengan kode kesalahan yang diperoleh dari GetLastError().

## Dapatkan Proses Saat Ini

GetCurrentProcess API di C++ adalah fungsi sederhana yang digunakan untuk menangani proses saat ini. Meskipun tidak memiliki aplikasi langsung dalam pengujian keamanan siber atau penetrasi, namun dapat digunakan untuk mengumpulkan informasi tentang proses yang sedang berjalan atau untuk melakukan operasi tertentu pada proses tersebut. Berikut ini contoh dasar cara menggunakan GetCurrentProcess:

In [ ]:
#include <windows.h>
#include <iostream>

int main() {
    HANDLE hProcess = GetCurrentProcess();

    if (hProcess == NULL) {
        std::cerr << "GetCurrentProcess failed. Error code: " << GetLastError() << std::endl;
        return 1;
    }

    std::cout << "Handle to the current process: " << hProcess << std::endl;

    // Do further operations with the process handle if needed

    // Close the handle when done with it
    CloseHandle(hProcess);

    return 0;
}

## BukaProcessToken


OpenProcessToken API di C++ biasanya digunakan dalam pengujian keamanan siber dan penetrasi ketika Anda perlu mendapatkan pegangan pada token akses yang terkait dengan suatu proses. Token akses berisi informasi tentang konteks keamanan pengguna, termasuk hak istimewa, grup, dan hak pengguna. Berikut ini contoh cara menggunakan OpenProcessToken:

In [ ]:
#include <windows.h>
#include <iostream>

int main() {
    DWORD processId; // Replace with the target process ID
    HANDLE hProcess, hToken;

    // Replace 'processId' with the PID of the target process
    processId = 1234; // Example PID

    // Open the target process with PROCESS_QUERY_INFORMATION access rights
    hProcess = OpenProcess(PROCESS_QUERY_INFORMATION, FALSE, processId);

    if (hProcess == NULL) {
        std::cerr << "Failed to open the target process. Error code: " << GetLastError() << std::endl;
        return 1;
    }

    // Open the access token associated with the target process
    if (!OpenProcessToken(hProcess, TOKEN_QUERY, &hToken)) {
        std::cerr << "OpenProcessToken failed. Error code: " << GetLastError() << std::endl;
        CloseHandle(hProcess);
        return 1;
    }

    // Use the access token as needed (e.g., querying privileges or groups)

    // Close the process and token handles when done
    CloseHandle(hToken);
    CloseHandle(hProcess);

    return 0;
}

## PencarianAkunSid

API LookupAccountSid di C++ digunakan untuk mengubah pengidentifikasi keamanan (SID) menjadi nama pengguna atau grup. API ini dapat membantu dalam pengujian keamanan siber dan penetrasi ketika Anda perlu mengidentifikasi pengguna atau grup yang terkait dengan SID. Berikut ini contoh cara menggunakan LookupAccountSid:

In [ ]:
#include <windows.h>
#include <iostream>
#include <sddl.h>

int main() {
    // Replace this string with the SID you want to look up
    LPCWSTR sidString = L"S-1-5-21-3623811015-3361044348-30300820-1013";

    PSID pSid = NULL;
    if (!ConvertStringSidToSidW(sidString, &pSid)) {
        std::cerr << "ConvertStringSidToSidW failed. Error code: " << GetLastError() << std::endl;
        return 1;
    }

    WCHAR szName[MAX_PATH];
    DWORD cchName = sizeof(szName) / sizeof(szName[0]);
    WCHAR szDomain[MAX_PATH];
    DWORD cchDomain = sizeof(szDomain) / sizeof(szDomain[0]);
    SID_NAME_USE sidType;

    if (LookupAccountSidW(NULL, pSid, szName, &cchName, szDomain, &cchDomain, &sidType)) {
        std::wcout << L"User/Group Name: " << szDomain << L"\\" << szName << std::endl;
    } else {
        std::cerr << "LookupAccountSidW failed. Error code: " << GetLastError() << std::endl;
    }

    LocalFree(pSid);

    return 0;
}

## KonversiSidToStringSidA

API ConvertSidToStringSidA di C++ digunakan untuk mengubah pengidentifikasi keamanan (SID) menjadi representasi stringnya. Hal ini dapat berguna dalam pengujian keamanan siber dan penetrasi ketika Anda perlu menampilkan atau memanipulasi SID dalam format yang dapat dibaca manusia. Berikut ini contoh cara menggunakan ConvertSidToStringSidA:

In [ ]:
#include <windows.h>
#include <iostream>

int main() {
    // Replace this string with the SID you want to convert
    const char* sidString = "S-1-5-21-3623811015-3361044348-30300820-1013";

    PSID pSid = NULL;
    if (!ConvertStringSidToSidA(sidString, &pSid)) {
        std::cerr << "ConvertStringSidToSidA failed. Error code: " << GetLastError() << std::endl;
        return 1;
    }

    LPSTR stringSid = NULL;
    if (ConvertSidToStringSidA(pSid, &stringSid)) {
        std::cout << "String representation of SID: " << stringSid << std::endl;
        LocalFree(stringSid); // Free the allocated memory
    } else {
        std::cerr << "ConvertSidToStringSidA failed. Error code: " << GetLastError() << std::endl;
    }

    LocalFree(pSid); // Free the SID structure

    return 0;
}

## Kotak PesanA

API MessageBoxA di C++ digunakan untuk menampilkan dialog kotak pesan pada sistem operasi Windows. Meskipun ini bukan alat langsung untuk pengujian keamanan siber atau penetrasi, kotak pesan dapat digunakan untuk berbagai tujuan, termasuk menampilkan peringatan atau informasi selama pengujian. Berikut ini contoh cara menggunakan MessageBoxA untuk menampilkan kotak pesan sederhana:

In [ ]:
#include <windows.h>

int main() {
    // Display a message box with a "Hello, World!" message
    MessageBoxA(NULL, "Hello, World!", "Message Box Example", MB_OK | MB_ICONINFORMATION);

    return 0;
}

## Kotak Pesan Terpikat

Membuat API HookedMessageBox dari awal akan melibatkan penerapan fungsi kotak pesan khusus dengan teknik pengait, yang bisa jadi cukup rumit. Hooking adalah teknik yang digunakan untuk mencegat dan mengubah perilaku fungsi yang ada. Ini adalah topik khusus di bidang pengembangan perangkat lunak, dan sering digunakan untuk debugging, pemantauan, atau penyesuaian perilaku sistem.

Di bawah ini adalah contoh sederhana tentang bagaimana Anda dapat menggunakan fungsi hooking untuk mencegat dan mengubah perilaku fungsi MessageBoxA. Perhatikan bahwa contoh ini menunjukkan konsep hooking dan tidak cocok untuk tujuan pengujian keamanan siber atau penetrasi:


In [ ]:
#include <windows.h>
#include <iostream>

// Function pointer type for the original MessageBoxA function
typedef int(WINAPI* MessageBoxAType)(HWND, LPCSTR, LPCSTR, UINT);

// Function pointer to store the address of the original MessageBoxA function
MessageBoxAType originalMessageBoxA;

// Custom MessageBoxA function that intercepts and modifies the behavior
int WINAPI CustomMessageBoxA(HWND hWnd, LPCSTR lpText, LPCSTR lpCaption, UINT uType) {
    // Modify the message or behavior here
    std::cout << "Intercepted MessageBoxA:" << std::endl;
    std::cout << "Text: " << lpText << std::endl;
    std::cout << "Caption: " << lpCaption << std::endl;

    // Call the original MessageBoxA function
    return originalMessageBoxA(hWnd, lpText, lpCaption, uType);
}

int main() {
    // Get the address of the original MessageBoxA function
    HMODULE user32Module = GetModuleHandle(L"user32.dll");
    if (user32Module != NULL) {
        originalMessageBoxA = reinterpret_cast<MessageBoxAType>(GetProcAddress(user32Module, "MessageBoxA"));
    }

    // Check if we successfully obtained the original function pointer
    if (originalMessageBoxA == NULL) {
        std::cerr << "Failed to obtain the address of MessageBoxA." << std::endl;
        return 1;
    }

    // Set our custom MessageBoxA function as the hook
    MessageBoxAType customMessageBoxA = CustomMessageBoxA;
    originalMessageBoxA = reinterpret_cast<MessageBoxAType>(
        SetWindowsHookEx(WH_CBT, reinterpret_cast<HOOKPROC>(customMessageBoxA), NULL, GetCurrentThreadId())
    );

    if (originalMessageBoxA == NULL) {
        std::cerr << "Failed to set the hook." << std::endl;
        return 1;
    }

    // Trigger a MessageBoxA call to see the interception
    MessageBoxA(NULL, "Hello, World!", "Original MessageBoxA", MB_OK);

    // Remove the hook
    UnhookWindowsHookEx(reinterpret_cast<HHOOK>(originalMessageBoxA));

    return 0;
}

Kami mendefinisikan fungsi MessageBoxA khusus (CustomMessageBoxA) yang memotong dan mengubah perilaku fungsi MessageBoxA asli. Dalam hal ini, ia hanya mencetak pesan dan keterangan ke konsol dan kemudian memanggil fungsi aslinya.

Kami memperoleh alamat fungsi MessageBoxA asli menggunakan GetProcAddress.

Kami menggunakan fungsi SetWindowsHookEx untuk mengatur fungsi kustom kami sebagai pengait untuk MessageBoxA. Ini memotong panggilan ke MessageBoxA dan mengarahkannya ke fungsi khusus kami.

Kami memanggil MessageBoxA untuk memicu hook dan mendemonstrasikan intersepsi.

Terakhir, kami menghapus hook menggunakan UnhookWindowsHookEx.

## Dapatkan AlamatProc

API GetProcAddress di C++ digunakan untuk mengambil alamat fungsi atau variabel yang diekspor dalam pustaka tautan dinamis (DLL) atau dapat dieksekusi (EXE). Ini dapat digunakan dalam pengujian keamanan siber dan penetrasi untuk memeriksa fungsi yang tersedia dan berpotensi menemukan kerentanan atau kelemahan dalam aplikasi target. Berikut ini contoh cara menggunakan GetProcAddress:

In [ ]:
#include <windows.h>
#include <iostream>

int main() {
    // Replace these values with the target DLL and function names
    const char* dllName = "user32.dll";
    const char* functionName = "MessageBoxA";

    // Load the target DLL
    HMODULE hModule = LoadLibraryA(dllName);

    if (hModule == NULL) {
        std::cerr << "Failed to load the DLL. Error code: " << GetLastError() << std::endl;
        return 1;
    }

    // Get the address of the function
    FARPROC functionAddress = GetProcAddress(hModule, functionName);

    if (functionAddress == NULL) {
        std::cerr << "Failed to get the address of the function. Error code: " << GetLastError() << std::endl;
        FreeLibrary(hModule);
        return 1;
    }

    std::cout << "Address of " << functionName << " in " << dllName << ": " << functionAddress << std::endl;

    // Free the loaded DLL
    FreeLibrary(hModule);

    return 0;
}

## BuatProsesA

CreateProcessA API di C++ biasanya digunakan untuk membuat proses baru. Ini dapat berguna dalam pengujian keamanan siber dan penetrasi ketika Anda perlu meluncurkan proses baru, seperti menjalankan alat eksternal atau menjalankan perintah pada sistem. Berikut ini contoh cara menggunakan CreateProcessA:

In [ ]:
#include <windows.h>
#include <iostream>

int main() {
    // Replace these values with the path to the executable and command-line arguments
    const char* executablePath = "C:\\Path\\To\\YourProgram.exe";
    const char* commandLineArgs = ""; // Optional command-line arguments

    // Structure for process information
    PROCESS_INFORMATION pi;
    
    // Structure for startup information
    STARTUPINFOA si;
    ZeroMemory(&si, sizeof(STARTUPINFOA));
    si.cb = sizeof(STARTUPINFOA);

    if (CreateProcessA(
        NULL,               // Use the application name from the command line
        (LPSTR)executablePath, // Path to the executable
        NULL,               // Process handle not inheritable
        NULL,               // Thread handle not inheritable
        FALSE,              // Set handle inheritance to FALSE
        0,                  // No creation flags
        NULL,               // Use parent's environment block
        NULL,               // Use parent's starting directory 
        &si,                // Pointer to STARTUPINFO structure
        &pi                 // Pointer to PROCESS_INFORMATION structure
    )) {
        std::cout << "Process created successfully!" << std::endl;
        std::cout << "Process ID: " << pi.dwProcessId << std::endl;
        
        // Close process and thread handles to avoid resource leaks
        CloseHandle(pi.hProcess);
        CloseHandle(pi.hThread);
    } else {
        std::cerr << "CreateProcessA failed. Error code: " << GetLastError() << std::endl;
        return 1;
    }

    return 0;
}

Ganti executablePath dengan jalur ke executable yang ingin Anda jalankan, dan commandLineArgs dengan argumen baris perintah opsional.

Kami mendefinisikan struktur STARTUPINFOA untuk memberikan informasi tentang bagaimana proses harus dimulai, dan struktur PROCESS_INFORMATION untuk menerima informasi tentang proses yang baru dibuat.

Kami memanggil CreateProcessA dengan jalur eksekusi dan argumen baris perintah yang ditentukan. Jika berhasil, ini akan membuat proses baru dan mengembalikan informasi tentangnya di struktur PROCESS_INFORMATION.

Kami mencetak ID proses (PID) dari proses yang baru dibuat ke konsol.

Terakhir, kami menutup proses dan pegangan thread untuk menghindari kebocoran sumber daya.

## Proses Terbuka

OpenProcess API di C++ digunakan untuk menangani proses yang ada. Ini dapat berguna dalam pengujian keamanan siber dan penetrasi ketika Anda perlu berinteraksi atau memanipulasi proses lain yang berjalan di sistem Windows. Berikut ini contoh cara menggunakan OpenProcess:

In [ ]:
#include <windows.h>
#include <iostream>

int main() {
    DWORD processId; // Replace with the target process ID
    HANDLE hProcess;

    // Replace 'processId' with the PID of the target process
    processId = 1234; // Example PID

    // Open the target process with PROCESS_QUERY_INFORMATION access rights
    hProcess = OpenProcess(PROCESS_QUERY_INFORMATION, FALSE, processId);

    if (hProcess == NULL) {
        std::cerr << "Failed to open the target process. Error code: " << GetLastError() << std::endl;
        return 1;
    }

    std::cout << "Successfully opened the target process with handle: " << hProcess << std::endl;

    // Perform operations on the target process as needed

    // Close the handle when done with it
    CloseHandle(hProcess);

    return 0;
}

## DuplikatHandle

DuplikatHandle API di C++ digunakan untuk menduplikasi pegangan ke objek seperti proses, thread, atau file. Ini dapat berguna dalam pengujian keamanan siber dan penetrasi ketika Anda perlu berbagi pegangan antar proses atau melakukan operasi tertentu pada pegangan duplikat tanpa mempengaruhi yang asli. Berikut ini contoh cara menggunakan DuplikatHandle:

In [ ]:
#include <windows.h>
#include <iostream>

int main() {
    HANDLE hProcess; // Replace with the source process handle
    HANDLE hDuplicateProcess = NULL;

    // Replace 'hProcess' with the source process handle you want to duplicate
    hProcess = OpenProcess(PROCESS_QUERY_INFORMATION, FALSE, GetCurrentProcessId());

    if (hProcess == NULL) {
        std::cerr << "Failed to open the source process. Error code: " << GetLastError() << std::endl;
        return 1;
    }

    // Duplicate the handle
    if (DuplicateHandle(GetCurrentProcess(), hProcess, GetCurrentProcess(), &hDuplicateProcess, 0, FALSE, DUPLICATE_SAME_ACCESS)) {
        std::cout << "Handle duplicated successfully!" << std::endl;
        
        // Perform operations using the duplicated handle (hDuplicateProcess) as needed

        // Close the duplicated handle when done with it
        CloseHandle(hDuplicateProcess);
    } else {
        std::cerr << "DuplicateHandle failed. Error code: " << GetLastError() << std::endl;
    }

    // Close the source process handle
    CloseHandle(hProcess);

    return 0;
}

Ganti hProcess dengan pegangan proses sumber yang ingin Anda duplikasi. Dalam contoh ini, kami menggunakan OpenProcess untuk mendapatkan pegangan dari proses saat ini sebagai contoh.

Kami memanggil OpenProcess untuk membuka proses sumber dengan hak akses PROCESS_QUERY_INFORMATION. Anda harus menyesuaikan hak akses dan mendapatkan pegangan proses sumber sesuai dengan kebutuhan spesifik Anda.

Kami memeriksa apakah fungsi OpenProcess berhasil menangani proses sumber. Jika gagal, kami mencetak pesan kesalahan dengan kode kesalahan dari GetLastError().

Kami menggunakan DuplikatHandle untuk menduplikasi pegangan proses sumber (hProcess) ke dalam proses saat ini (GetCurrentProcess()). Pegangan duplikat disimpan di hDuplicationProcess.

Jika DuplikatHandle berhasil, maka pegangan tersebut akan diduplikasi, dan kita dapat menggunakan pegangan yang diduplikasi (hDuplicationProcess) untuk melakukan operasi pada proses sumber sesuai kebutuhan.

Terakhir, kami menutup pegangan proses sumber (hProcess) dan pegangan duplikat (hDuplicationProcess) setelah selesai menggunakannya untuk melepaskan sumber daya terkait.


## VirtualAllocEx

VirtualAllocEx API di C++ digunakan untuk mengalokasikan memori dalam ruang alamat proses tertentu. Hal ini dapat berguna dalam pengujian keamanan siber dan penetrasi ketika Anda perlu mengalokasikan memori dalam proses lain untuk berbagai tujuan, seperti injeksi kode atau analisis memori. Berikut ini contoh cara menggunakan VirtualAllocEx:

In [ ]:
#include <windows.h>
#include <iostream>

int main() {
    HANDLE hProcess; // Replace with the target process handle
    LPVOID lpBaseAddress = NULL; // Request any available address
    SIZE_T dwSize = 4096; // Allocate 4 KB (adjust as needed)
    DWORD flAllocationType = MEM_COMMIT | MEM_RESERVE;
    DWORD flProtect = PAGE_EXECUTE_READWRITE; // Adjust protection as needed

    // Replace 'hProcess' with the target process handle you want to allocate memory in
    hProcess = OpenProcess(PROCESS_ALL_ACCESS, FALSE, 1234); // Replace '1234' with the target process ID

    if (hProcess == NULL) {
        std::cerr << "Failed to open the target process. Error code: " << GetLastError() << std::endl;
        return 1;
    }

    LPVOID lpRemoteBuffer = VirtualAllocEx(hProcess, lpBaseAddress, dwSize, flAllocationType, flProtect);

    if (lpRemoteBuffer == NULL) {
        std::cerr << "VirtualAllocEx failed. Error code: " << GetLastError() << std::endl;
        CloseHandle(hProcess);
        return 1;
    }

    std::cout << "Memory allocated successfully in the target process at address: " << lpRemoteBuffer << std::endl;

    // Perform operations using the allocated memory in the target process as needed

    // Free the allocated memory when done
    VirtualFreeEx(hProcess, lpRemoteBuffer, 0, MEM_RELEASE);

    // Close the target process handle
    CloseHandle(hProcess);

    return 0;
}

## VirtualProtectEx

VirtualProtectEx API di C++ digunakan untuk mengubah atribut perlindungan wilayah memori dalam ruang alamat proses tertentu. Hal ini dapat berguna dalam pengujian keamanan siber dan penetrasi ketika Anda perlu mengubah atribut perlindungan memori dalam proses lain untuk berbagai tujuan, seperti injeksi kode atau manipulasi memori. Berikut ini contoh cara menggunakan VirtualProtectEx:

In [ ]:
#include <windows.h>
#include <iostream>

int main() {
    HANDLE hProcess; // Replace with the target process handle
    LPVOID lpAddress = nullptr; // Address of the memory region to protect
    SIZE_T dwSize = 4096; // Size of the memory region (adjust as needed)
    DWORD flNewProtect = PAGE_EXECUTE_READWRITE; // New protection attributes

    // Replace 'hProcess' with the target process handle you want to modify memory protection in
    hProcess = OpenProcess(PROCESS_ALL_ACCESS, FALSE, 1234); // Replace '1234' with the target process ID

    if (hProcess == NULL) {
        std::cerr << "Failed to open the target process. Error code: " << GetLastError() << std::endl;
        return 1;
    }

    if (VirtualProtectEx(hProcess, lpAddress, dwSize, flNewProtect, nullptr)) {
        std::cout << "Memory protection attributes modified successfully." << std::endl;

        // Perform operations on the protected memory as needed

        // Restore the original protection attributes if necessary
        DWORD flOldProtect;
        VirtualProtectEx(hProcess, lpAddress, dwSize, flOldProtect, nullptr);
    } else {
        std::cerr << "VirtualProtectEx failed. Error code: " << GetLastError() << std::endl;
    }

    // Close the target process handle
    CloseHandle(hProcess);

    return 0;
}

## Setel Konteks Benang

SetThreadContext API di C++ digunakan untuk mengatur konteks thread tertentu, yang mencakup nilai register dan flag. Hal ini dapat berguna dalam pengujian keamanan siber dan penetrasi untuk berbagai tujuan, seperti memodifikasi perilaku thread atau mengubah alur eksekusi dalam proses target. Namun, penting untuk diingat bahwa penggunaan API ini untuk tujuan yang tidak sah atau berbahaya dapat menimbulkan implikasi hukum dan etika yang serius. Berikut ini contoh cara menggunakan SetThreadContext:

In [ ]:
#include <windows.h>
#include <iostream>

int main() {
    DWORD processId; // Replace with the target process ID
    HANDLE hProcess, hThread;

    // Replace 'processId' with the PID of the target process
    processId = 1234; // Example PID

    // Open the target process with PROCESS_ALL_ACCESS access rights
    hProcess = OpenProcess(PROCESS_ALL_ACCESS, FALSE, processId);

    if (hProcess == NULL) {
        std::cerr << "Failed to open the target process. Error code: " << GetLastError() << std::endl;
        return 1;
    }

    // Open a thread within the target process (e.g., the primary thread)
    hThread = OpenThread(THREAD_ALL_ACCESS, FALSE, GetCurrentThreadId()); // Replace with the target thread ID

    if (hThread == NULL) {
        std::cerr << "Failed to open the target thread. Error code: " << GetLastError() << std::endl;
        CloseHandle(hProcess);
        return 1;
    }

    // Define a CONTEXT structure to store the thread context
    CONTEXT context;
    context.ContextFlags = CONTEXT_FULL; // Retrieve full context

    // Get the current context of the target thread
    if (!GetThreadContext(hThread, &context)) {
        std::cerr << "GetThreadContext failed. Error code: " << GetLastError() << std::endl;
        CloseHandle(hThread);
        CloseHandle(hProcess);
        return 1;
    }

    // Modify the context as needed
    // For example, you can change register values or flags in the 'context' structure here

    // Set the modified context back to the target thread
    if (!SetThreadContext(hThread, &context)) {
        std::cerr << "SetThreadContext failed. Error code: " << GetLastError() << std::endl;
        CloseHandle(hThread);
        CloseHandle(hProcess);
        return 1;
    }

    std::cout << "Thread context modified successfully." << std::endl;

    // Close handles when done
    CloseHandle(hThread);
    CloseHandle(hProcess);

    return 0;
}

## Antrian PenggunaAPC

QueueUserAPC (Asynchronous Procedure Call) API di C++ digunakan untuk mengantri fungsi yang ditentukan pengguna untuk dieksekusi dalam ruang alamat thread tertentu. Ini dapat berguna dalam pengujian keamanan siber dan penetrasi ketika Anda perlu memasukkan dan mengeksekusi kode dalam proses target untuk berbagai tujuan. Namun, perlu diketahui bahwa memanipulasi proses jarak jauh tanpa izin yang sesuai dapat menimbulkan implikasi hukum dan etika yang serius. Berikut ini contoh cara menggunakan QueueUserAPC:

In [ ]:
#include <windows.h>
#include <iostream>

// Define a custom APC function to be executed within the target thread
VOID CALLBACK CustomAPCFunction(ULONG_PTR dwParam) {
    // Code to be executed within the target thread
    std::cout << "Custom APC function executed within the target thread." << std::endl;
}

int main() {
    DWORD processId; // Replace with the target process ID
    HANDLE hProcess, hThread;

    // Replace 'processId' with the PID of the target process
    processId = 1234; // Example PID

    // Open the target process with PROCESS_ALL_ACCESS access rights
    hProcess = OpenProcess(PROCESS_ALL_ACCESS, FALSE, processId);

    if (hProcess == NULL) {
        std::cerr << "Failed to open the target process. Error code: " << GetLastError() << std::endl;
        return 1;
    }

    // Open a thread within the target process (e.g., the primary thread)
    hThread = OpenThread(THREAD_ALL_ACCESS, FALSE, GetCurrentThreadId()); // Replace with the target thread ID

    if (hThread == NULL) {
        std::cerr << "Failed to open the target thread. Error code: " << GetLastError() << std::endl;
        CloseHandle(hProcess);
        return 1;
    }

    // Queue the custom APC function to be executed within the target thread
    if (QueueUserAPC(CustomAPCFunction, hThread, 0)) {
        std::cout << "APC function queued successfully." << std::endl;

        // Trigger the APC by suspending and resuming the target thread
        SuspendThread(hThread);
        ResumeThread(hThread);
    } else {
        std::cerr << "QueueUserAPC failed. Error code: " << GetLastError() << std::endl;
    }

    // Close handles when done
    CloseHandle(hThread);
    CloseHandle(hProcess);

    return 0;
}

## BuatRemoteThread

CreateRemoteThread API di C++ digunakan untuk membuat thread baru dalam ruang alamat proses jarak jauh tertentu, memungkinkan Anda memasukkan dan mengeksekusi kode dalam proses tersebut. Hal ini dapat berguna dalam pengujian keamanan siber dan penetrasi ketika Anda perlu memanipulasi atau menganalisis perilaku proses target. Namun, perlu diketahui bahwa memanipulasi proses jarak jauh tanpa izin yang sesuai dapat menimbulkan implikasi hukum dan etika yang serius. Berikut ini contoh cara menggunakan CreateRemoteThread:

In [ ]:
#include <windows.h>
#include <iostream>

int main() {
    DWORD processId; // Replace with the target process ID
    HANDLE hProcess;

    // Replace 'processId' with the PID of the target process
    processId = 1234; // Example PID

    // Open the target process with PROCESS_ALL_ACCESS access rights
    hProcess = OpenProcess(PROCESS_ALL_ACCESS, FALSE, processId);

    if (hProcess == NULL) {
        std::cerr << "Failed to open the target process. Error code: " << GetLastError() << std::endl;
        return 1;
    }

    // Define the code to be executed within the target process
    // In this example, we create a simple thread that displays a message box
    const char* codeToInject = R"(
        #include <windows.h>
        int main() {
            MessageBoxA(NULL, "Injected Code", "Injection Example", MB_ICONINFORMATION);
            return 0;
        }
    )";

    // Allocate memory within the target process for the code
    LPVOID pRemoteCode = VirtualAllocEx(hProcess, NULL, strlen(codeToInject) + 1, MEM_COMMIT, PAGE_EXECUTE_READWRITE);

    if (pRemoteCode == NULL) {
        std::cerr << "Failed to allocate memory in the target process. Error code: " << GetLastError() << std::endl;
        CloseHandle(hProcess);
        return 1;
    }

    // Write the code to the allocated memory
    if (!WriteProcessMemory(hProcess, pRemoteCode, codeToInject, strlen(codeToInject) + 1, NULL)) {
        std::cerr << "Failed to write code to the target process. Error code: " << GetLastError() << std::endl;
        VirtualFreeEx(hProcess, pRemoteCode, 0, MEM_RELEASE);
        CloseHandle(hProcess);
        return 1;
    }

    // Create a remote thread within the target process to execute the code
    HANDLE hRemoteThread = CreateRemoteThread(hProcess, NULL, 0, (LPTHREAD_START_ROUTINE)pRemoteCode, NULL, 0, NULL);

    if (hRemoteThread == NULL) {
        std::cerr << "Failed to create a remote thread in the target process. Error code: " << GetLastError() << std::endl;
        VirtualFreeEx(hProcess, pRemoteCode, 0, MEM_RELEASE);
        CloseHandle(hProcess);
        return 1;
    }

    std::cout << "Remote thread created successfully." << std::endl;

    // Wait for the remote thread to finish
    WaitForSingleObject(hRemoteThread, INFINITE);

    // Close handles when done
    CloseHandle(hRemoteThread);
    VirtualFreeEx(hProcess, pRemoteCode, 0, MEM_RELEASE);
    CloseHandle(hProcess);

    return 0;
}